In [237]:
# Importar las librerías para manejar y analizar datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [238]:
import pandas as pd

base_dir = "C:\\Users\\marim\\Downloads"

carpetas_meses = [
    "Ene_2024", "Febrero_2024", "Marzo_2024", "Abril_2024", "Mayo_2024", "Junio_2024",
    "Julio_2024", "Agosto_2024", "Septiembre_2024", "Octubre_2024", "Noviembre_2024", "Diciembre_2024"
]

# Nombres de los archivos
archivos = {
    "ocupados": "Ocupados.CSV",
    "otras_formas_trabajo": "Otras formas de trabajo.CSV",
    "otros_ingresos": "Otros ingresos e impuestos.CSV",
    "caracteristicas": "Características generales, seguridad social en salud y educación.CSV",
    "hogar": "Datos del hogar y la vivienda.CSV",
    "fuerza": "Fuerza de trabajo.CSV",
}

# Diccionario para guardar los DataFrames por mes y tipo
dataframes_por_mes = {}

for carpeta in carpetas_meses:
    mes_key = carpeta.replace(" ", "_")
    ruta_csv = os.path.join(base_dir, carpeta, "CSV")
    dataframes_por_mes[mes_key] = {}

    for nombre_archivo, archivo in archivos.items():
        ruta_archivo = os.path.join(ruta_csv, archivo)

        try:
            df = pd.read_csv(ruta_archivo, sep=";", encoding="latin1", low_memory=False)
            dataframes_por_mes[mes_key][nombre_archivo] = df
            print(f"Cargado: {mes_key} -> {nombre_archivo} ({df.shape[0]} filas)")
        except FileNotFoundError:
            print(f"Archivo no encontrado: {mes_key} -> {nombre_archivo}")
        except Exception as e:
            print(f"Error en {mes_key} -> {nombre_archivo}: {e}")

Cargado: Ene_2024 -> ocupados (28815 filas)
Cargado: Ene_2024 -> otras_formas_trabajo (55738 filas)
Cargado: Ene_2024 -> otros_ingresos (55738 filas)
Cargado: Ene_2024 -> caracteristicas (70648 filas)
Cargado: Ene_2024 -> hogar (24247 filas)
Cargado: Ene_2024 -> fuerza (55738 filas)
Cargado: Febrero_2024 -> ocupados (29943 filas)
Cargado: Febrero_2024 -> otras_formas_trabajo (56856 filas)
Cargado: Febrero_2024 -> otros_ingresos (56856 filas)
Cargado: Febrero_2024 -> caracteristicas (72148 filas)
Cargado: Febrero_2024 -> hogar (24743 filas)
Cargado: Febrero_2024 -> fuerza (56856 filas)
Cargado: Marzo_2024 -> ocupados (29638 filas)
Cargado: Marzo_2024 -> otras_formas_trabajo (56063 filas)
Cargado: Marzo_2024 -> otros_ingresos (56063 filas)
Cargado: Marzo_2024 -> caracteristicas (70719 filas)
Cargado: Marzo_2024 -> hogar (24647 filas)
Cargado: Marzo_2024 -> fuerza (56063 filas)
Cargado: Abril_2024 -> ocupados (29741 filas)
Cargado: Abril_2024 -> otras_formas_trabajo (55696 filas)
Cargado:

In [239]:
llaves = ['DIRECTORIO', 'SECUENCIA_P', 'ORDEN']

# Crear un diccionario para guardar el df final de cada mes
df_por_mes = {}

for mes, data in dataframes_por_mes.items():
    try:
        print(f"\nProcesando merges para: {mes}")

        # Asignar cada dataframe del mes
        df_fuerza = data.get("fuerza")
        df_ingresos = data.get("otros_ingresos")
        df_otras = data.get("otras_formas_trabajo")
        df_ocup = data.get("ocupados")
        df_caract = data.get("caracteristicas")
        df_hogar = data.get("hogar")

        if df_fuerza is None or df_ingresos is None:
            print(f"No se puede procesar {mes} por falta de datos clave (fuerza o ingresos)")
            continue

        # 1. Merge fuerza + ingresos
        cols_fuerza = [col for col in df_fuerza.columns if col not in df_ingresos.columns or col in llaves]
        fuerza_ingresos = df_fuerza[cols_fuerza].merge(df_ingresos, on=llaves, how='outer')

        # 2. Otras formas
        if df_otras is not None:
            cols_otras = [col for col in df_otras.columns if col not in fuerza_ingresos.columns or col in llaves]
            df_unido = fuerza_ingresos.merge(df_otras[cols_otras], on=llaves, how='outer')
        else:
            df_unido = fuerza_ingresos

        # 3. Ocupados
        if df_ocup is not None:
            cols_ocup = [col for col in df_ocup.columns if col not in df_unido.columns or col in llaves]
            df_unido = df_unido.merge(df_ocup[cols_ocup], on=llaves, how='left')

        # 4. Características
        if df_caract is not None:
            cols_caract = [col for col in df_caract.columns if col not in df_unido.columns or col in llaves]
            df_unido = df_unido.merge(df_caract[cols_caract], on=llaves, how='left')

        # 5. Hogar
        if df_hogar is not None:
            cols_hogar = [col for col in df_hogar.columns if col not in df_unido.columns or col == 'DIRECTORIO']
            df_unido = df_unido.merge(df_hogar[cols_hogar], on='DIRECTORIO', how='left')

        # Guardar DataFrame final del mes
        df_por_mes[mes] = df_unido
        print(f"{mes} → {df_unido.shape[0]} personas, {df_unido.shape[1]} columnas")

    except Exception as e:
        print(f"Error en el procesamiento de {mes}: {e}")



Procesando merges para: Ene_2024
Ene_2024 → 56839 personas, 469 columnas

Procesando merges para: Febrero_2024
Febrero_2024 → 58127 personas, 469 columnas

Procesando merges para: Marzo_2024
Marzo_2024 → 57093 personas, 469 columnas

Procesando merges para: Abril_2024
Abril_2024 → 56532 personas, 469 columnas

Procesando merges para: Mayo_2024
Mayo_2024 → 56761 personas, 469 columnas

Procesando merges para: Junio_2024
Junio_2024 → 56469 personas, 469 columnas

Procesando merges para: Julio_2024
Julio_2024 → 55891 personas, 469 columnas

Procesando merges para: Agosto_2024
Agosto_2024 → 55649 personas, 469 columnas

Procesando merges para: Septiembre_2024
Septiembre_2024 → 55117 personas, 469 columnas

Procesando merges para: Octubre_2024
Octubre_2024 → 54311 personas, 469 columnas

Procesando merges para: Noviembre_2024
Noviembre_2024 → 53419 personas, 469 columnas

Procesando merges para: Diciembre_2024
Diciembre_2024 → 52319 personas, 469 columnas


In [240]:
# Paso 1: Agregar columna MES a cada df del diccionario
for mes, df in df_por_mes.items():
    df["MES"] = mes  # puedes estandarizar si quieres solo el nombre del mes

# Paso 2: Concatenar todos los DataFrames en uno solo
df_2024 = pd.concat(df_por_mes.values(), ignore_index=True)

# Resultado final
print(f"df_2024 tiene {df_2024.shape[0]} registros y {df_2024.shape[1]} columnas.")

df_2024 tiene 668527 registros y 469 columnas.


In [241]:
# Crear un subconjunto del DataFrame con las columnas especificadas
df_2024 = df_2024[[
    'DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'FT', 'FFT', 'PET', 'DPTO', 'AREA',
    "P6040", "P6030S1", "P6030S3", "P3271", "P3039", "P3038",
    "P6080", "P2057", "P6080S1", "P6070", "P6050", "P6083", "P6081", "P6071", "P6008",
    "P6160", "P6170", "P3041", "P6090", "P6100", "P6120",
    "P6500", "OFICIO_C8", "P6800", "P6430", "P6440", "P6450", "P6460",
    "P6422", "P6424S1", "P6424S2", "P6424S3", "P6424S5", "P6426", "P6630S1", "P6240",
    "RAMA2D_R4", "RAMA4D_R4",
    "P3077S1", "P3077S1A1", "P3077S1A2", "P3077S2", "P3077S2A1", "P3077S2A2",
    "P3077S3", "P3077S3A1", "P3077S3A2", "P3079S1", "P3079S1A1", "P3079S1A2",
    "P3079S2", "P3079S2A1", "P3079S2A2", "P3079S3", "P3079S3A1", "P3079S3A2",
    "P3081S1", "P3081S1A1", "P3081S1A2", "P3081S2", "P3081S2A1", "P3081S2A2",
    "P3081S3", "P3081S3A1", "P3081S3A2", "P3082S1", "P3082S1A1", "P3082S1A2",
    "P3082S2", "P3082S2A1", "P3082S2A2", "P3082S3", "P3082S3A1", "P3082S3A2",
    "P4000", "P4030S1A1", "P5090", "CLASE", "P5110", "P5140", "P4030S1", "P4030S2",
    "P4030S3", "P4030S4", "P4030S5", "P5050",
    "P3147S1", "P3147S2", "P3147S3", "P3147S4", "P3147S5", "P3147S6", "P3147S7",
    "P3147S8", "P3147S9", "P3147S11", "P7500S2", "P7500S3", "P7510S3", "P3042"
]]

# Mostrar el tamaño del DataFrame
print("Tamaño DataFrame: ", df_2024.shape)


Tamaño DataFrame:  (668527, 108)


In [242]:
# Diccionario original de variables a renombrar
nombres_variables = {

    # Demográficas
    "P6040": "EDAD",
    "P6030S1": "MES_NACIMIENTO",
    "P6030S3": "AÑO_NACIMIENTO",

    "P3271": "SEXO_NACIMIENTO",
    "P3039": "GENERO",
    "P3038": "ATRACCION_POR",

    # Etnia
    "P6080": "ETNIA",
    "P2057": "CAMPESINO",
    "P6080S1": "GRUPO_INDIGENA",

    # Relaciones
    "P6070": "ESTADO_CIVIL",
    "P6050": "PARENTESCO_CON_JEFE_HOGAR",
    "P6083": "MADRE_RESIDE_HOGAR",
    "P6081": "PADRE_RESIDE_HOGAR",
    "P6071": "CONYUGE_RESIDE_HOGAR",
    "P6008": "TOTAL_PERSONAS_HOGAR",

    # Educación
    "P6160": "SABE_LEER",
    "P6170": "ACTUALMENTE_ESTUDIA",
    "P3041": "ACTUALMENTE_INSTITUCION",
    "P3042": "MAXIMO_NIVEL_EDUCATIVO",

    # Salud
    "P6090": "ENTIDAD_SEGURIDAD_SOCIAL_SALUD",
    "P6100": "REGIMEN_SEGURIDAD_SOCIAL_SALUD",
    "P6120": "PAGO_SALUD",

    # Trabajo
    "P6500": "INGRESO",
    "OFICIO_C8": "OCUPACION",
    "P6800": "HORAS_TRABAJO",
    "P6430": "POSICION_OCUPACIONAL",
    "P6440": "EXISTENCIA_CONTRATO",
    "P6450": "CONTRATO_VERBAL_ESCRITO",
    "P6460": "TERMINO_CONTRATO",
    "P6422": "CONFORME_TIPO_CONTRATO",
    "P6424S1": "VACACIONES_SUELDO",
    "P6424S2": "PRIMA_NAVIDAD",
    "P6424S3": "CESANTIA",
    "P6424S5": "LICENCIA_ENFERMEDAD_PAGADA",
    "P6426": "TIEMPO_TRABAJANDO_EMPRESA_ACTUAL",
    "P6630S1": "PRIMA_SERVICIOS",
    "P6240": "ACTIVIDAD_MAYOR_TIEMPO",
    "RAMA2D_R4": "SECTOR_AMPLIO",
    "RAMA4D_R4": "SECTOR_DETALLADO",

    # Otras formas de trabajo:

    # LIMPIEZA DEL HOGAR
    "P3077S1": "LIMPIEZA_EN_SU_HOGAR",
    "P3077S1A1": "LIMPIEZA_EN_SU_HOGAR_DIAS",
    "P3077S1A2": "LIMPIEZA_EN_SU_HOGAR_HORAS_DIA",
    "P3077S2": "LIMPIEZA_HOGAR_FAMILIARES",
    "P3077S2A1": "LIMPIEZA_HOGAR_FAMILIARES_DIAS",
    "P3077S2A2": "LIMPIEZA_HOGAR_FAMILIARES_HORAS_DIA",
    "P3077S3": "LIMPIEZA_HOGAR_NO_FAMILIARES",
    "P3077S3A1": "LIMPIEZA_HOGAR_NO_FAMILIARES_DIAS",
    "P3077S3A2": "LIMPIEZA_HOGAR_NO_FAMILIARES_HORAS_DIA",

    # CUIDADO DE NIÑOS
    "P3079S1": "CUIDADO_NINOS_SU_HOGAR",
    "P3079S1A1": "CUIDADO_NINOS_SU_HOGAR_DIAS",
    "P3079S1A2": "CUIDADO_NINOS_SU_HOGAR_HORAS_DIA",
    "P3079S2": "CUIDADO_NINOS_HOGAR_FAMILIARES",
    "P3079S2A1": "CUIDADO_NINOS_HOGAR_FAMILIARES_DIAS",
    "P3079S2A2": "CUIDADO_NINOS_HOGAR_FAMILIARES_HORAS_DIA",
    "P3079S3": "CUIDADO_NINOS_HOGAR_NO_FAMILIARES",
    "P3079S3A1": "CUIDADO_NINOS_HOGAR_NO_FAMILIARES_DIAS",
    "P3079S3A2": "CUIDADO_NINOS_HOGAR_NO_FAMILIARES_HORAS_DIA",

    # CUIDADO DE MAYORES
    "P3081S1": "CUIDADO_MAYORES_SU_HOGAR",
    "P3081S1A1": "CUIDADO_MAYORES_SU_HOGAR_DIAS",
    "P3081S1A2": "CUIDADO_MAYORES_SU_HOGAR_HORAS_DIA",
    "P3081S2": "CUIDADO_MAYORES_HOGAR_FAMILIARES",
    "P3081S2A1": "CUIDADO_MAYORES_HOGAR_FAMILIARES_DIAS",
    "P3081S2A2": "CUIDADO_MAYORES_HOGAR_FAMILIARES_HORAS_DIA",
    "P3081S3": "CUIDADO_MAYORES_HOGAR_NO_FAMILIARES",
    "P3081S3A1": "CUIDADO_MAYORES_HOGAR_NO_FAMILIARES_DIAS",
    "P3081S3A2": "CUIDADO_MAYORES_HOGAR_NO_FAMILIARES_HORAS_DIA",

    # APOYO ESCOLAR
    "P3082S1": "APOYO_TAREAS_SU_HOGAR",
    "P3082S1A1": "APOYO_TAREAS_SU_HOGAR_DIAS",
    "P3082S1A2": "APOYO_TAREAS_SU_HOGAR_HORAS_DIA",
    "P3082S2": "APOYO_TAREAS_HOGAR_FAMILIARES",
    "P3082S2A1": "APOYO_TAREAS_HOGAR_FAMILIARES_DIAS",
    "P3082S2A2": "APOYO_TAREAS_HOGAR_FAMILIARES_HORAS_DIA",
    "P3082S3": "APOYO_TAREAS_HOGAR_NO_FAMILIARES",
    "P3082S3A1": "APOYO_TAREAS_HOGAR_NO_FAMILIARES_DIAS",
    "P3082S3A2": "APOYO_TAREAS_HOGAR_NO_FAMILIARES_HORAS_DIA",

    # Vivienda y ubicación
    "P4000": "TIPO_VIVIENDA",
    "P4030S1A1": "ESTRATO",
    "P5090": "TENENCIA_VIVIENDA",
    "CLASE": "URBANO_RURAL",
    "P5110": "VALOR_VIVIENDA",
    "P5140": "VALOR_ARRIENDO",
    "P4030S1": "ELECTRICIDAD",
    "P4030S2": "GAS_NATURAL",
    "P4030S3": "ALCANTARILLADO",
    "P4030S4": "RECOLECCION_BASURA",
    "P4030S5": "ACUEDUCTO",
    "P5050": "AGUA_POTABLE",


    # Problemas debido a la pandemia
    "P3147S1": "TUVO_COVID",
    "P3147S2": "DIFICULTAD_ALIMENTOS_PRODUCTOS_LIMPIEZA",
    "P3147S3": "NO_PAGOS",
    "P3147S4": "REDUCCION_INGRESOS",
    "P3147S5": "NO_TRABAJO",
    "P3147S6": "SUSPENSION_SIN_REMUNERACION",
    "P3147S7": "DESPIDO",
    "P3147S8": "SUSPENSION_CLASES_PRESENCIALES",
    "P3147S9": "SENTIMIENTOS_NEGATIVOS",
    "P3147S11": "SIN_DIFICULTADES",

    # No ocupados
    "P7250": "SEMANAS_BUSCANDO_TRABAJO",

    # Otros ingresos
    "P7500S2": "PENSION",
    "P7500S3": "PENSION_SEPARACION",
    "P7510S3": "AYUDA_DINERO_INSTITUCION"

}

# Renombrar las columnas del subconjunto
df_2024 = df_2024.rename(columns=nombres_variables)

In [243]:
df_2024.head(20)

,DIRECTORIO,SECUENCIA_P,ORDEN,FT,FFT,PET,DPTO,AREA,EDAD,MES_NACIMIENTO,...,NO_TRABAJO,SUSPENSION_SIN_REMUNERACION,DESPIDO,SUSPENSION_CLASES_PRESENCIALES,SENTIMIENTOS_NEGATIVOS,SIN_DIFICULTADES,PENSION,PENSION_SEPARACION,AYUDA_DINERO_INSTITUCION,MAXIMO_NIVEL_EDUCATIVO
0,7655976,1,1,NaN,1.0,1,5,5.0,39,5.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,5.0
1,7655976,1,2,1.0,NaN,1,5,5.0,32,2.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,5.0
2,7655977,1,1,1.0,NaN,1,5,5.0,39,9.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,4.0
3,7655977,1,2,1.0,NaN,1,5,5.0,22,5.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,4.0
4,7655979,1,1,NaN,1.0,1,5,5.0,67,5.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,NaN,3.0
5,7655979,1,2,1.0,NaN,1,5,5.0,37,4.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,10.0
6,7655980,1,1,1.0,NaN,1,5,5.0,41,8.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0
7,7655980,1,2,1.0,NaN,1,5,5.0,31,10.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,4.0
8,7655980,1,3,NaN,1.0,1,5,5.0,15,12.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,4.0
9,7655981,1,1,1.0,NaN,1,5,5.0,48,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0


In [244]:
# Mostrar los tipos de datos de las columnas
df_2024.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668527 entries, 0 to 668526
Data columns (total 108 columns):
 #    Column                                         Dtype  
---   ------                                         -----  
 0    DIRECTORIO                                     int64  
 1    SECUENCIA_P                                    int64  
 2    ORDEN                                          int64  
 3    FT                                             float64
 4    FFT                                            float64
 5    PET                                            int64  
 6    DPTO                                           int64  
 7    AREA                                           float64
 8    EDAD                                           int64  
 9    MES_NACIMIENTO                                 float64
 10   AÑO_NACIMIENTO                                 float64
 11   SEXO_NACIMIENTO                                int64  
 12   GENERO                      

In [245]:
# Calcular el porcentaje de valores faltantes en cada columna
missing_percent = (df_2024.isnull().sum()/df_2024.shape[0]*100).sort_values(ascending=False)
print(missing_percent)

APOYO_TAREAS_HOGAR_NO_FAMILIARES_HORAS_DIA       99.951984
APOYO_TAREAS_HOGAR_NO_FAMILIARES_DIAS            99.951984
CUIDADO_MAYORES_HOGAR_NO_FAMILIARES_HORAS_DIA    99.938222
CUIDADO_MAYORES_HOGAR_NO_FAMILIARES_DIAS         99.938222
CUIDADO_NINOS_HOGAR_NO_FAMILIARES_HORAS_DIA      99.928051
                                                   ...    
ELECTRICIDAD                                      0.000000
TENENCIA_VIVIENDA                                 0.000000
ACUEDUCTO                                         0.000000
AGUA_POTABLE                                      0.000000
MAXIMO_NIVEL_EDUCATIVO                            0.000000
Length: 108, dtype: float64


In [246]:
# Calcular el porcentaje de no nulos por columna
no_nulos = (df_2024.notnull().sum() / df_2024.shape[0]) * 100

# Filtrar solo las columnas con 100% de completitud
variables_completas = no_nulos[no_nulos == 100.0].index.tolist()

# Imprimir resultado
print("🔍 Variables con 100% de valores no nulos:")
for var in variables_completas:
    print(var)



🔍 Variables con 100% de valores no nulos:
DIRECTORIO
SECUENCIA_P
ORDEN
PET
DPTO
EDAD
SEXO_NACIMIENTO
ETNIA
CAMPESINO
ESTADO_CIVIL
PARENTESCO_CON_JEFE_HOGAR
MADRE_RESIDE_HOGAR
PADRE_RESIDE_HOGAR
TOTAL_PERSONAS_HOGAR
SABE_LEER
ACTUALMENTE_ESTUDIA
ENTIDAD_SEGURIDAD_SOCIAL_SALUD
ACTIVIDAD_MAYOR_TIEMPO
LIMPIEZA_EN_SU_HOGAR
LIMPIEZA_HOGAR_FAMILIARES
LIMPIEZA_HOGAR_NO_FAMILIARES
CUIDADO_NINOS_SU_HOGAR
CUIDADO_NINOS_HOGAR_FAMILIARES
CUIDADO_NINOS_HOGAR_NO_FAMILIARES
CUIDADO_MAYORES_SU_HOGAR
CUIDADO_MAYORES_HOGAR_FAMILIARES
CUIDADO_MAYORES_HOGAR_NO_FAMILIARES
APOYO_TAREAS_SU_HOGAR
APOYO_TAREAS_HOGAR_FAMILIARES
APOYO_TAREAS_HOGAR_NO_FAMILIARES
TIPO_VIVIENDA
TENENCIA_VIVIENDA
URBANO_RURAL
ELECTRICIDAD
GAS_NATURAL
ALCANTARILLADO
RECOLECCION_BASURA
ACUEDUCTO
AGUA_POTABLE
MAXIMO_NIVEL_EDUCATIVO


In [247]:
# Crear un subconjunto del DataFrame con las columnas especificadas
df_ingreso = df_2024[[
    'DIRECTORIO', 'SECUENCIA_P', 'ORDEN',
    'INGRESO', 'FT', 'PET','EDAD', 'SEXO_NACIMIENTO', 'GENERO', 'ATRACCION_POR', 'ETNIA', 'CAMPESINO',
    'ESTRATO', 'TIPO_VIVIENDA', 'TENENCIA_VIVIENDA', 'URBANO_RURAL', 'ELECTRICIDAD', 'GAS_NATURAL', 'DPTO', 'ACUEDUCTO', 'ALCANTARILLADO', 'RECOLECCION_BASURA',
    'AGUA_POTABLE', 'TOTAL_PERSONAS_HOGAR', 'MAXIMO_NIVEL_EDUCATIVO', 'SABE_LEER', 'ENTIDAD_SEGURIDAD_SOCIAL_SALUD',
    'OCUPACION', 'HORAS_TRABAJO', 'POSICION_OCUPACIONAL', 'EXISTENCIA_CONTRATO', "CONTRATO_VERBAL_ESCRITO", 'TERMINO_CONTRATO', 'CONFORME_TIPO_CONTRATO', 'VACACIONES_SUELDO',
    'PRIMA_NAVIDAD','CESANTIA', 'LICENCIA_ENFERMEDAD_PAGADA', 'TIEMPO_TRABAJANDO_EMPRESA_ACTUAL','PRIMA_SERVICIOS', 'SECTOR_AMPLIO', 'SECTOR_DETALLADO', 'ACTUALMENTE_ESTUDIA',
    'ACTIVIDAD_MAYOR_TIEMPO', 'GRUPO_INDIGENA', 'LIMPIEZA_EN_SU_HOGAR', 'CUIDADO_NINOS_SU_HOGAR', 'CUIDADO_MAYORES_SU_HOGAR', 'APOYO_TAREAS_SU_HOGAR', 'LIMPIEZA_EN_SU_HOGAR_DIAS',
    'LIMPIEZA_EN_SU_HOGAR_HORAS_DIA', 'CUIDADO_MAYORES_SU_HOGAR_DIAS', 'CUIDADO_MAYORES_SU_HOGAR_HORAS_DIA', 'CUIDADO_NINOS_SU_HOGAR_DIAS','CUIDADO_NINOS_SU_HOGAR_HORAS_DIA'
]]

# Mostrar el tamaño del DataFrame
print("Tamaño DataFrame: ", df_ingreso.shape)

Tamaño DataFrame:  (668527, 55)


In [248]:
df_ingreso = df_ingreso[df_ingreso['INGRESO'].notnull()]

print("Tamaño DataFrame con ingresos no nulos:", df_ingreso.shape)

Tamaño DataFrame con ingresos no nulos: (186771, 55)


In [249]:
# Mostrar los tipos de datos de las columnas
df_ingreso.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 186771 entries, 1 to 668525
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   DIRECTORIO                          186771 non-null  int64  
 1   SECUENCIA_P                         186771 non-null  int64  
 2   ORDEN                               186771 non-null  int64  
 3   INGRESO                             186771 non-null  float64
 4   FT                                  186771 non-null  float64
 5   PET                                 186771 non-null  int64  
 6   EDAD                                186771 non-null  int64  
 7   SEXO_NACIMIENTO                     186771 non-null  int64  
 8   GENERO                              185675 non-null  float64
 9   ATRACCION_POR                       185675 non-null  float64
 10  ETNIA                               186771 non-null  int64  
 11  CAMPESINO                      

In [250]:
# Calcular el porcentaje de valores faltantes en cada columna
missing_percent = (df_ingreso.isnull().sum()/df_ingreso.shape[0]*100).sort_values(ascending=False)
print(missing_percent)

CUIDADO_MAYORES_SU_HOGAR_HORAS_DIA    98.115339
CUIDADO_MAYORES_SU_HOGAR_DIAS         98.115339
GRUPO_INDIGENA                        97.493187
CUIDADO_NINOS_SU_HOGAR_HORAS_DIA      90.802105
CUIDADO_NINOS_SU_HOGAR_DIAS           90.802105
LIMPIEZA_EN_SU_HOGAR_DIAS             40.159875
LIMPIEZA_EN_SU_HOGAR_HORAS_DIA        40.159875
TERMINO_CONTRATO                      33.877850
GENERO                                 0.586815
ATRACCION_POR                          0.586815
ESTRATO                                0.261818
ETNIA                                  0.000000
CAMPESINO                              0.000000
TIPO_VIVIENDA                          0.000000
TENENCIA_VIVIENDA                      0.000000
EDAD                                   0.000000
PET                                    0.000000
FT                                     0.000000
SEXO_NACIMIENTO                        0.000000
INGRESO                                0.000000
DIRECTORIO                             0

In [251]:
df_ingreso[['TERMINO_CONTRATO', 'ESTRATO']] = df_ingreso[['TERMINO_CONTRATO', 'ESTRATO']].fillna(9)
df_ingreso = df_ingreso.dropna(subset=['GENERO', 'ATRACCION_POR'])
df_ingreso['GRUPO_INDIGENA'] = df_ingreso['GRUPO_INDIGENA'].fillna(0)

condicion = (df_ingreso['LIMPIEZA_EN_SU_HOGAR'] == 2) & (df_ingreso['LIMPIEZA_EN_SU_HOGAR_DIAS'].isna())
df_ingreso.loc[condicion, 'LIMPIEZA_EN_SU_HOGAR_DIAS'] = 0

condicion = (df_ingreso['LIMPIEZA_EN_SU_HOGAR'] == 2) & (df_ingreso['LIMPIEZA_EN_SU_HOGAR_HORAS_DIA'].isna())
df_ingreso.loc[condicion, 'LIMPIEZA_EN_SU_HOGAR_HORAS_DIA'] = 0

condicion = (df_ingreso['CUIDADO_MAYORES_SU_HOGAR'] == 2) & (df_ingreso['CUIDADO_MAYORES_SU_HOGAR_DIAS'].isna())
df_ingreso.loc[condicion, 'CUIDADO_MAYORES_SU_HOGAR_DIAS'] = 0

condicion = (df_ingreso['CUIDADO_MAYORES_SU_HOGAR'] == 2) & (df_ingreso['CUIDADO_MAYORES_SU_HOGAR_HORAS_DIA'].isna())
df_ingreso.loc[condicion, 'CUIDADO_MAYORES_SU_HOGAR_HORAS_DIA'] = 0

condicion = (df_ingreso['CUIDADO_NINOS_SU_HOGAR'] == 2) & (df_ingreso['CUIDADO_NINOS_SU_HOGAR_DIAS'].isna())
df_ingreso.loc[condicion, 'CUIDADO_NINOS_SU_HOGAR_DIAS'] = 0

condicion = (df_ingreso['CUIDADO_NINOS_SU_HOGAR'] == 2) & (df_ingreso['CUIDADO_NINOS_SU_HOGAR_HORAS_DIA'].isna())
df_ingreso.loc[condicion, 'CUIDADO_NINOS_SU_HOGAR_HORAS_DIA'] = 0

# Mostrar los tipos de datos de las columnas
df_ingreso.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
Index: 185675 entries, 1 to 668525
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   DIRECTORIO                          185675 non-null  int64  
 1   SECUENCIA_P                         185675 non-null  int64  
 2   ORDEN                               185675 non-null  int64  
 3   INGRESO                             185675 non-null  float64
 4   FT                                  185675 non-null  float64
 5   PET                                 185675 non-null  int64  
 6   EDAD                                185675 non-null  int64  
 7   SEXO_NACIMIENTO                     185675 non-null  int64  
 8   GENERO                              185675 non-null  float64
 9   ATRACCION_POR                       185675 non-null  float64
 10  ETNIA                               185675 non-null  int64  
 11  CAMPESINO                      

In [252]:
valores_pet = df_ingreso['ACTIVIDAD_MAYOR_TIEMPO'].unique()
print( valores_pet)

[1 4 6 3 2]


In [253]:
df_ingreso.head()

,DIRECTORIO,SECUENCIA_P,ORDEN,INGRESO,FT,PET,EDAD,SEXO_NACIMIENTO,GENERO,ATRACCION_POR,...,LIMPIEZA_EN_SU_HOGAR,CUIDADO_NINOS_SU_HOGAR,CUIDADO_MAYORES_SU_HOGAR,APOYO_TAREAS_SU_HOGAR,LIMPIEZA_EN_SU_HOGAR_DIAS,LIMPIEZA_EN_SU_HOGAR_HORAS_DIA,CUIDADO_MAYORES_SU_HOGAR_DIAS,CUIDADO_MAYORES_SU_HOGAR_HORAS_DIA,CUIDADO_NINOS_SU_HOGAR_DIAS,CUIDADO_NINOS_SU_HOGAR_HORAS_DIA
1,7655976,1,2,1160000.0,1.0,1,32,2,2.0,1.0,...,1,1,2,2,1.0,2.0,0.0,0.0,7.0,6.0
2,7655977,1,1,1600000.0,1.0,1,39,1,1.0,2.0,...,1,2,2,2,2.0,2.0,0.0,0.0,0.0,0.0
3,7655977,1,2,1500000.0,1.0,1,22,1,1.0,2.0,...,1,2,2,2,2.0,1.0,0.0,0.0,0.0,0.0
6,7655980,1,1,1300000.0,1.0,1,41,1,1.0,2.0,...,1,2,2,2,1.0,1.0,0.0,0.0,0.0,0.0
7,7655980,1,2,1300000.0,1.0,1,31,2,2.0,1.0,...,1,2,2,2,2.0,2.0,0.0,0.0,0.0,0.0


In [254]:
columnas_categoricas = [
    'FT', 'PET', 'SEXO_NACIMIENTO', 'GENERO', 'ATRACCION_POR', 'ETNIA', 'GRUPO_INDIGENA', 'CAMPESINO',
    'ESTRATO', 'TIPO_VIVIENDA', 'TENENCIA_VIVIENDA', 'URBANO_RURAL', 'ELECTRICIDAD',
    'GAS_NATURAL', 'DPTO', 'ACUEDUCTO', 'ALCANTARILLADO', 'RECOLECCION_BASURA',
    'AGUA_POTABLE', 'MAXIMO_NIVEL_EDUCATIVO', 'SABE_LEER', 'ENTIDAD_SEGURIDAD_SOCIAL_SALUD',
    'OCUPACION', 'POSICION_OCUPACIONAL', 'EXISTENCIA_CONTRATO', 'CONTRATO_VERBAL_ESCRITO',
    'TERMINO_CONTRATO', 'CONFORME_TIPO_CONTRATO', 'VACACIONES_SUELDO', 'PRIMA_NAVIDAD',
    'CESANTIA', 'LICENCIA_ENFERMEDAD_PAGADA', 'PRIMA_SERVICIOS', 'SECTOR_AMPLIO', 'SECTOR_DETALLADO', 'ACTUALMENTE_ESTUDIA',
    'ACTIVIDAD_MAYOR_TIEMPO', 'LIMPIEZA_EN_SU_HOGAR', 'CUIDADO_NINOS_SU_HOGAR', 'CUIDADO_MAYORES_SU_HOGAR', 'APOYO_TAREAS_SU_HOGAR'
]

for col in columnas_categoricas:
    df_ingreso[col] = df_ingreso[col].astype('category')


In [255]:
# Mostrar los tipos de datos de las columnas
df_ingreso.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 185675 entries, 1 to 668525
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   DIRECTORIO                          185675 non-null  int64   
 1   SECUENCIA_P                         185675 non-null  int64   
 2   ORDEN                               185675 non-null  int64   
 3   INGRESO                             185675 non-null  float64 
 4   FT                                  185675 non-null  category
 5   PET                                 185675 non-null  category
 6   EDAD                                185675 non-null  int64   
 7   SEXO_NACIMIENTO                     185675 non-null  category
 8   GENERO                              185675 non-null  category
 9   ATRACCION_POR                       185675 non-null  category
 10  ETNIA                               185675 non-null  category
 11  CAMPESINO         

In [256]:
df_ingreso[['GENERO', 'SEXO_NACIMIENTO', 'LIMPIEZA_EN_SU_HOGAR']].head(20)

# 2 es mujer
# 1 parece ser si, 2 no

,GENERO,SEXO_NACIMIENTO,LIMPIEZA_EN_SU_HOGAR
1,2.0,2,1
2,1.0,1,1
3,1.0,1,1
6,1.0,1,1
7,2.0,2,1
12,2.0,2,1
13,2.0,2,1
18,1.0,1,2
20,1.0,1,2
22,1.0,1,1
